In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import re

# Set up Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # run in headless mode
chrome_options.add_argument("--no-sandbox")  # bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # overcome limited resource problems
chrome_options.add_argument("--disable-gpu")  # applicable to windows os only
chrome_options.add_argument("start-maximized")  # open Browser in maximized mode
chrome_options.add_argument("disable-infobars")  # disabling infobars
chrome_options.add_argument("--disable-extensions")  # disabling extensions


def scraperLoop(key, text_list, driver, numberOfObjects, time_of_waiting = 4):
    
    time.sleep(1)
    chat_xpath = '/html/body/div/div[1]/div/div[2]/div/form/div/div/textarea'
    chat_input = driver.find_element(By.XPATH, chat_xpath)
    chat_input.send_keys(f'Could you give me {numberOfObjects} synonyms of {key} in Mandarin? Without any spaces. Just words. No yapping')
    chat_input.send_keys(Keys.RETURN)
    time.sleep(time_of_waiting)
    for i in range(20):
        try: 
            fetch_xpath = '/html/body/div/div[1]/div/div[1]/div/div[2]/div[1]'
            fetch_output = driver.find_element(By.XPATH, fetch_xpath)
            text_list[key] = re.split(r"[.\s,，。]+", fetch_output.text)
            break
        except Exception as e:
            time.sleep(1)
    new_chat_xpath = '/html/body/div/div[1]/nav[3]/div[1]/a[2]'
    new_chat_entry = driver.find_element(By.XPATH, new_chat_xpath)
    new_chat_entry.click()

In [36]:

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
time.sleep(1)
driver.get("https://twllm.com/")

b1_xpath = '/html/body/div[2]/div/div/div/div/button'
b1_input = driver.find_element(By.XPATH, b1_xpath)
b1_input.click()

text_key = ['好', '不好', '大']
text_list = {}

for key in text_key:
    
    scraperLoop(key, text_list, driver, 10)
driver.quit()


In [37]:
text_list


{'好': ['好的', '非常好', '很好', '不錯', '棒極了', '太好了', '挺好', '相當好', '出色地'],
 '不好': ['不好', '差', '不佳', '欠佳', '低劣', '糟糕', '不如意', '不理想', '有缺陷', '有問題'],
 '大': ['大', '巨大', '宏大', '浩大', '廣闊', '寬敞', '高大', '龐大', 'vastness', 'mammoth']}